In [1]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from statsmodels import api as sm
from statsmodels.tsa.stattools import coint
from statsmodels.stats.multitest import multipletests
from tqdm import tqdm

plt.style.use('ggplot')

# Test for cointegrated pairs from 50 simulated stocks

In [2]:
sig = 0.05   # significance threshold
n_obs = 365  # 1 year of simulated daily data
n_stocks = 50

n_comparisons = int(np.ceil((n_stocks * (n_stocks - 1)) / 2))  # 50 choose 2

false_positives = 0

def generate_random_i1_series(n_obs, n_series):
    return np.random.randn(n_obs * n_series).reshape((n_obs, n_series)).cumsum(axis=0) * np.random.randn(n_series)

A = generate_random_i1_series(n_obs, n_comparisons)
B = generate_random_i1_series(n_obs, n_comparisons)

pvalues = np.zeros(n_comparisons)
for i in tqdm(range(n_comparisons)):
    a = A[:, i]
    b = B[:, i]
    
    t, pvalue, crits = coint(A[:, i], B[:, i])
    pvalues[i] = pvalue

100%|██████████████████████████████████████████████████████████████████████████████| 1225/1225 [01:06<00:00, 18.30it/s]


In [3]:
print(f'Number of false positives from {n_comparisons} comparisons (unadjusted p-values): {(pvalues < sig).sum():d}')

Number of false positives from 1225 comparisons (unadjusted p-values): 51


# Adjust p-values using the Holm-Sidak step-down method

In [4]:
reject, pvalues_corrected, alpha_sidak, alpha_bonf = multipletests(pvalues, method='holm-sidak', alpha=sig)

In [5]:
print(f'Number of false positives from {n_comparisons} comparisons (adjusted p-values): {(pvalues_corrected < sig).sum():d}')

Number of false positives from 1225 comparisons (adjusted p-values): 0
